imports

In [1]:

### open lib
import pandas as pd
import os

# Spark
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, TimestampType
from pyspark.sql.functions import col, desc

In [2]:
### self defined lib
from ingestion_lib.analyse import parse_dataset, infer_column_type, add_key_column, dupe_check
from ingestion_lib.main import records, categories, tbl_parsed_dict

Loaded categories
Loaded records
categories
	 filename :  categories
	 columns :  ['category_key', 'type', 'payment_category', 'subcategory', 'item', 'nature', 'color', 'category']
	 data_type :  ['string', 'string', 'string', 'string', 'string', 'string', 'string', 'string']
	 pk :  category_key
	 check_pk :  PASS
	 check_null_columns :  subcategory, item, nature
records
	 filename :  records
	 columns :  ['record_key', 'category_key', 'account', 'category', 'currency', 'amount', 'ref_currency_amount', 'type', 'payment_type', 'payment_type_local', 'note', 'date', 'gps_latitude', 'gps_longitude', 'gps_accuracy_in_meters', 'warranty_in_month', 'transfer', 'payee', 'labels', 'envelope_id', 'custom_category']
	 data_type :  ['string', 'string', 'string', 'string', 'string', 'float', 'float', 'string', 'string', 'string', 'string', 'string', 'float', 'float', 'float', 'int', 'bool', 'string', 'float', 'int', 'bool']
	 pk :  record_key
	 check_pk :  PASS
	 check_null_columns :  note, gps_la

Setup local SparkSession

In [ ]:
# Ensures proper Python worker binding
os.environ["PYSPARK_PYTHON"] = "python"
os.environ["PYSPARK_DRIVER_PYTHON"] = "python"
os.environ["SPARK_LOCAL_IP"] = "127.0.0.1"

# Point to the JDBC driver
jdbc_jar_path = "C:\\Program Files\\Java\\sqljdbc_12.10\\enu\\jars\\mssql-jdbc.jar"

# Start Spark session with JDBC driver
spark = SparkSession.builder \
    .appName("SQLServerAccess") \
    .config("spark.driver.bindAddress", "127.0.0.1") \
    .config("spark.driver.host", "localhost") \
    .config("spark.network.timeout", "100s") \
    .config("spark.python.worker.reuse", "false") \
    .config("spark.jars", jdbc_jar_path) \
    .getOrCreate()



Ingest as Spark df

In [4]:
categories_df = spark.createDataFrame(categories)
categories_df.show()

+--------------------+--------+----------------+--------------------+----+------+-------+--------------------+
|        category_key|    type|payment_category|         subcategory|item|nature|  color|            category|
+--------------------+--------+----------------+--------------------+----+------+-------+--------------------+
|Expenses:Food & D...|Expenses|   Food & Drinks|                 NaN| NaN|  Need|#FF5722|       Food & Drinks|
|Expenses:Restaura...|Expenses|   Food & Drinks|    Restaurants, bar| NaN|  Need|#FF5722|    Restaurants, bar|
|  Expenses:Groceries|Expenses|   Food & Drinks|           Groceries| NaN|  Need|#FF5722|           Groceries|
|  Expenses:Fast-food|Expenses|   Food & Drinks|           Fast-food| NaN|  Need|#FF5722|           Fast-food|
|   Expenses:Shopping|Expenses|        Shopping|                 NaN| NaN|  Want|#4FC3F7|            Shopping|
|Expenses:Clothes ...|Expenses|        Shopping|     Clothes & shoes| NaN|  Want|#4FC3F7|     Clothes & shoes|
|

In [5]:
records_df = spark.createDataFrame(records)
records_df.show()

+--------------------+--------------------+-------+--------------------+--------+--------+-------------------+--------+--------------+------------------+--------------------+-------------------+------------+-------------+----------------------+-----------------+--------+------+------+-----------+---------------+
|          record_key|        category_key|account|            category|currency|  amount|ref_currency_amount|    type|  payment_type|payment_type_local|                note|               date|gps_latitude|gps_longitude|gps_accuracy_in_meters|warranty_in_month|transfer| payee|labels|envelope_id|custom_category|
+--------------------+--------------------+-------+--------------------+--------+--------+-------------------+--------+--------------+------------------+--------------------+-------------------+------------+-------------+----------------------+-----------------+--------+------+------+-----------+---------------+
|2025-03-31 23:31:...|  Expenses:Fast-food|Savings|       

In [6]:
records_df = records_df.withColumn('timestamp', col('date').cast(TimestampType()))
records_df = records_df.select('record_key', 'category_key', 'account', 'category', 'amount', 'ref_currency_amount', 'type','timestamp','transfer')

categories_df = categories_df.select('category_key', 'type', 'payment_category', 'subcategory', 'item', 'nature').distinct()

Validation for categories

In [43]:
categories_df.select('type').distinct().show()
categories_df.select('nature').distinct().show()

+--------+
|    type|
+--------+
|Expenses|
|  Income|
+--------+

+------+
|nature|
+------+
|  Want|
|  Need|
|  Must|
|   NaN|
+------+



In [42]:
categories_df.select('payment_category').distinct().show()

+--------------------+
|    payment_category|
+--------------------+
|            Shopping|
|       Food & Drinks|
|             Housing|
|      Transportation|
|             Vehicle|
|Life & Entertainment|
|   Communication, PC|
|  Financial expenses|
|         Investments|
|              Income|
|            TRANSFER|
|              Others|
+--------------------+



In [67]:
categories_df.where('payment_category is null').count()

0

Validation for records

In [53]:
records_df.select('type').distinct().show()
records_df.select('transfer').distinct().show()
records_df.select('account').distinct().show()

+--------+
|    type|
+--------+
|  Income|
|Expenses|
+--------+

+--------+
|transfer|
+--------+
|    true|
|   false|
+--------+

+-------+
|account|
+-------+
|Savings|
|  Zerdh|
|   Card|
|   Cash|
| Equity|
| Salary|
+-------+



In [59]:
print(records_df.count())
print(records_df.where('transfer is True').count())

1893
294


In [60]:
records_df.select('category').distinct().count()

65

In [66]:
records_df.groupBy('category').count().orderBy(desc('count')).show()

+--------------------+-----+
|            category|count|
+--------------------+-----+
|            TRANSFER|  294|
|           Fast-food|  242|
|                Fuel|  121|
|     Loan, interests|   85|
|    Lending, renting|   77|
|             Missing|   62|
|           Groceries|   59|
|    Restaurants, bar|   57|
|Books, audio, sub...|   55|
|   Health and beauty|   46|
|      Charity, gifts|   44|
|Telephone, mobile...|   43|
|              Income|   42|
|       Food & Drinks|   37|
|     Clothes & shoes|   36|
|             Parking|   33|
|           Auto/taxi|   30|
|Software, apps, g...|   26|
|Holiday, trips, h...|   26|
|                  RD|   24|
+--------------------+-----+
only showing top 20 rows



Validation for joining

In [70]:
join_df = records_df.join(categories_df, how='left').where('payment_category is null')
print(join_df.count())

0


In [ ]:
join_df.groupBy('payment_category').agg('amount').show()

+----------------+-----------+
|payment_category|sum(amount)|
+----------------+-----------+
+----------------+-----------+



Validation for datatypes

In [7]:
print('\ncategories\n')
categories_col_dict = {field.name: str(field.dataType).replace('Type()','') for field in categories_df.schema.fields}
for col, type in categories_col_dict.items():
    print(col, ' : ', type)
    
print('\nrecords\n')
records_col_dict = {field.name: str(field.dataType).replace('Type()','') for field in records_df.schema.fields}
for col, type in records_col_dict.items():
    print(col, ' : ', type)


categories

category_key  :  String
type  :  String
payment_category  :  String
subcategory  :  String
item  :  String
nature  :  String

records

record_key  :  String
category_key  :  String
account  :  String
category  :  String
amount  :  Double
ref_currency_amount  :  Double
type  :  String
timestamp  :  Timestamp
transfer  :  Boolean


Write to database

In [ ]:
# SQL Server connection config
jdbc_url = (
    "jdbc:sqlserver://localhost:1433;"
    "databaseName=sqldb_staging;"
    "integratedSecurity=true;"
    "encrypt=true;"
    "trustServerCertificate=true"
)

connection_props = {
    "driver": "com.microsoft.sqlserver.jdbc.SQLServerDriver"
}

+----+----+
|col1|col2|
+----+----+
+----+----+



In [ ]:
categories_df.write.jdbc(url=jdbc_url, table='category', mode="overwrite", properties=connection_props)
records_df.write.jdbc(url=jdbc_url, table='records', mode="overwrite", properties=connection_props)

In [11]:
# Read a table
df = spark.read.jdbc(url=jdbc_url, table="dbo.category", properties=connection_props)
df.show()

+--------------------+--------+----------------+--------------------+----+------+
|        category_key|    type|payment_category|         subcategory|item|nature|
+--------------------+--------+----------------+--------------------+----+------+
|  Expenses:Fast-food|Expenses|   Food & Drinks|           Fast-food| NaN|  Need|
|   Expenses:Shopping|Expenses|        Shopping|                 NaN| NaN|  Want|
|Expenses:Restaura...|Expenses|   Food & Drinks|    Restaurants, bar| NaN|  Need|
|Expenses:Clothes ...|Expenses|        Shopping|     Clothes & shoes| NaN|  Want|
|Expenses:Food & D...|Expenses|   Food & Drinks|                 NaN| NaN|  Need|
|Expenses:Electron...|Expenses|        Shopping|Electronics, acce...| NaN|  Want|
|Expenses:Drug-sto...|Expenses|        Shopping| Drug-store, chemist| NaN|  Want|
|  Expenses:Groceries|Expenses|   Food & Drinks|           Groceries| NaN|  Need|
|Expenses:Pets, an...|Expenses|        Shopping|       Pets, animals| NaN|  Want|
|Expenses:Health

In [12]:
# Read a table
df = spark.read.jdbc(url=jdbc_url, table="dbo.records", properties=connection_props)
df.show()

+--------------------+--------------------+-------+--------------------+--------+-------------------+--------+-------------------+--------+
|          record_key|        category_key|account|            category|  amount|ref_currency_amount|    type|          timestamp|transfer|
+--------------------+--------------------+-------+--------------------+--------+-------------------+--------+-------------------+--------+
|2023-07-08 19:21:...|       Expenses:Fuel|Savings|                Fuel|  -540.0|             -540.0|Expenses|2023-07-08 19:21:52|   false|
|2023-07-07 19:30:...|Expenses:Food & D...|Savings|       Food & Drinks|   -40.0|              -40.0|Expenses|2023-07-07 19:30:58|   false|
|2023-07-02 10:56:...|       Expenses:Fuel|Savings|                Fuel|  -400.0|             -400.0|Expenses|2023-07-02 10:56:22|   false|
|2023-07-02 10:56:...|Expenses:Restaura...|Savings|    Restaurants, bar|   -80.0|              -80.0|Expenses|2023-07-02 10:56:14|   false|
|2023-07-02 10:55:..

In [13]:
spark.stop
# spark2.stop

<bound method SparkSession.stop of <pyspark.sql.session.SparkSession object at 0x0000020130229F90>>